In [1]:
print("Hello")

Hello


In [ ]:
from playwright.sync_api import sync_playwright, TimeoutError
import json
import time

def scrape_recipe(url, max_retries=3):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        )
        page = context.new_page()
        
        for attempt in range(max_retries):
            try:
                # Navigate to the URL
                page.goto(url, wait_until="networkidle")
                
                # Wait for key elements to be loaded
                page.wait_for_selector("h1", timeout=10000)
                
                recipe = {
                    "title": page.query_selector("h1").inner_text().strip(),
                    "url": url,
                    "ingredients": [],
                    "instructions": [],
                    "metadata": {}
                }
                
                # Get ingredients
                if ingredients_list := page.query_selector_all(".ingredients-list li"):
                    recipe["ingredients"] = [item.inner_text().strip() 
                                          for item in ingredients_list]
                
                # Get instructions
                if instructions_list := page.query_selector_all(".instructions-list li"):
                    recipe["instructions"] = [step.inner_text().strip() 
                                           for step in instructions_list]
                
                # Get metadata (cook time, servings, etc.)
                for meta_item in page.query_selector_all(".recipe-metadata"):
                    key = meta_item.get_attribute("data-field")
                    value = meta_item.inner_text().strip()
                    if key and value:
                        recipe["metadata"][key] = value
                
                return recipe
                
            except TimeoutError:
                if attempt < max_retries - 1:
                    print(f"Timeout occurred. Retrying... ({attempt + 1}/{max_retries})")
                    time.sleep(2)
                    continue
                else:
                    print("Max retries reached. Could not scrape recipe.")
                    return None
                    
            except Exception as e:
                print(f"Error occurred: {e}")
                return None
            
            finally:
                browser.close()

def save_recipe(recipe_data, filename="recipe.json"):
    if recipe_data:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(recipe_data, f, indent=4, ensure_ascii=False)
        print(f"Recipe successfully saved to {filename}!")
    else:
        print("No recipe data to save.")

# Example usage
if __name__ == "__main__":
    url = "YOUR_RECIPE_URL_HERE"
    recipe_data = scrape_recipe(url)
    save_recipe(recipe_data)